In [1]:
from transformers import SpeechT5Model, SpeechT5FeatureExtractor

import numpy as np
import torch
from datasets import load_dataset
from IPython.display import Audio, display

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
class CommonVoiceDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        audio_array = self.dataset[idx]['audio']['array']
        sampling_rate = self.dataset[idx]['audio']['sampling_rate']
        sentence = self.dataset[idx]['sentence']

        audio_tensor = torch.tensor(audio_array).float()
        return audio_tensor, sampling_rate, sentence

In [3]:
# Load dataset
dataset = load_dataset("mozilla-foundation/common_voice_11_0", "en", split='test')
train_dataset = CommonVoiceDataset(dataset)

Found cached dataset common_voice_11_0 (/export/home/lium/bdos/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/en/11.0.0/3f27acf10f303eac5b6fbbbe02495aeddb46ecffdb0a2fe3507fcfbf89094631)
